# Advent of Code 2023

generic import

In [2]:
import importlib.util
import logging
import numpy as np
import polars as pl

from logging import DEBUG, INFO
from pathlib import Path

VERBOSE = 5

parent_dir = Path().resolve()

# configure root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)
consoleHandler = logging.StreamHandler()
formatter = logging.Formatter('%(message)s')
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)

def set_log_level(log_level=INFO):
    logger = logging.getLogger()
    logger.setLevel(log_level)

def get_input_path(day, test=None, parent_dir=parent_dir):
    day_str = str(day).zfill(2)
    input_path = parent_dir.joinpath(f'day{day_str}/koen/data/input.txt')
    log_level = INFO
    if test:
        input_path = input_path.with_name(input_path.stem + f'_test{test}.txt')
        log_level = DEBUG
    set_log_level(log_level)
    return input_path

def path_import(absolute_path):
    spec = importlib.util.spec_from_file_location(absolute_path.stem, absolute_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def dynamic_load(day):
    day_str = str(day).zfill(2)
    module_path = parent_dir.joinpath(f'day{day_str}/koen/day{day_str}.py')
    return path_import(module_path)
    

## Day 01: Trebuchet?!

sql extravaganza in duckdb using the [DBeaver extension](https://duckdb.org/docs/archive/0.9.2/guides/sql_editors/dbeaver).

## Day 02: Cub Conundrum

crawling back to python

In [ ]:
day = 2
day2 = dynamic_load(day)
input_path = get_input_path(day=day, test=None)

### part 1

In [ ]:
max_red = 12
max_green = 13
max_blue = 14
df = day2.parse_input(input_path)
df

In [ ]:
df = (
    df
    .with_columns(
        pl.col('blue').list.max().alias('blue_max'),
        pl.col('red').list.max().alias('red_max'),
        pl.col('green').list.max().alias('green_max')
    )
)
df

In [ ]:
(
    df
    .filter(
        pl.col('blue_max').le(max_blue)
        & pl.col('red_max').le(max_red)
        & pl.col('green_max').le(max_green)
    )
    .select(
        pl.col('game').sum()
    )
)

### part 2

In [ ]:
(
    df
    .select(
        (
            pl.col('blue_max') * pl.col('red_max') * pl.col('green_max')
        ).sum()
    )
)

## Day 03: Gear Ratios

In [ ]:
day = 3
day3 = dynamic_load(day)
input_path = get_input_path(day=day, test=None)

### part 1

In [ ]:
numbers, symbol_df = day3.parse_input(input_path)

In [ ]:
sum([x.value for x in numbers if x.next_to_symbol(symbol_df)])

### part 2

In [ ]:
gear_candidate_df = (
    symbol_df
    .filter(pl.col('symbol') == '*')
    .with_columns(
        pl.concat_str(pl.col('row'), pl.col('column'), separator='-').alias('gear_id')
    )
)

In [ ]:
adjacent_parts = [x for x in numbers if x.next_to_symbol(gear_candidate_df)]
adjacent_gears = [(x.id, x.value, x.get_adjacent_gears(gear_candidate_df)) for x in adjacent_parts]

In [ ]:
gear_df = pl.DataFrame(adjacent_gears, schema=['part_id', 'part_value', 'gear_id'], orient='row').explode('gear_id')

In [ ]:
(
    gear_df
    .group_by('gear_id')
    .agg(
        pl.col('part_id').n_unique(),
        pl.col('part_value').product()
    )
    .filter(
        pl.col('part_id') == 2
    )
    .select(
        pl.col('part_value')
    ).sum()
)

## Day 04: Scratchcards 

In [ ]:
day = 4
day4 = dynamic_load(day)
input_path = get_input_path(day=day, test=None)

### part 1

In [ ]:
df = day4.parse_input(input_path)

In [ ]:
(
    df
    .with_columns(
        pl.col('winning').list.set_intersection(pl.col('ours')).list.len().alias('n_winning')
    )
    .select(
        pl.when(pl.col('n_winning') > 0)
        .then(2 ** (pl.col('n_winning') - 1))
        .otherwise(pl.lit(0))
        .cast(pl.Int64)
        .sum()
    )
)

### part 2

In [ ]:
df = (
    df
    .with_columns(
        pl.col('winning').list.set_intersection(pl.col('ours')).list.len().alias('n_winning')
    )
)

In [ ]:
card_new_cards = {x['card']: list(np.arange(x['n_winning']) + x['card'] + 1) for x in df.iter_rows(named=True)}

In [ ]:
card_copies = {x: 1 for x in card_new_cards.keys()}
for card in card_new_cards.keys():
    n_copies = card_copies.get(card, 0)
    new_cards = card_new_cards.get(card) * n_copies
    for new_card in new_cards:
        card_copies[new_card] = card_copies.get(new_card, 0) + 1

In [ ]:
sum(card_copies.values())

## Day 05: If You Give A Seed A Fertilizer

In [ ]:
day = 5
day5 = dynamic_load(day)
input_path = get_input_path(day=day, test=None)

### part 1

In [ ]:
seeds, mapping = day5.parse_input(input_path)

In [ ]:
mapping_graph = day5.MappingGraph(mapping, logger=logger)

In [ ]:
locations = mapping_graph.do_mapping('seed', seeds)
min(locations)

### part 2
need some time to ponder upon this one...

In [ ]:
seed_ranges = [(seeds[i], seeds[i] + seeds[i + 1] - 1) for i in range(0, len(seeds), 2)]
seed_ranges.sort(key=lambda x: x[0])
location_ranges = mapping_graph.do_range_mapping('seed', seed_ranges)

In [ ]:
min([min(x) for x in location_ranges])

## Day 06: Wait For It

In [39]:
day = 6
day6 = dynamic_load(day)
input_path = get_input_path(day=day, test=None)

### part 1

In [26]:
df = day6.parse_input(input_path)

In [27]:
(
    df
    .with_columns(
        pl.arange(1, df.height + 1).alias('id'),
        pl.int_ranges(0, 'max_time').alias('press_time')
    )
    .explode('press_time')
    .with_columns(
        (pl.col('press_time') * (pl.col('max_time') - pl.col('press_time'))).alias('distance')
    )
    .filter(pl.col('distance') > pl.col('distance_record'))
    .group_by('id')
    .count()
    .select(
        pl.col('count').product()
    )
)

count
i64
4403592


### part 2

In [40]:
time, distance = day6.parse_input2(input_path)

d = t x (m - t)  
-t^2 + mt = d  
t^2 -mt + d = 0  

ax^2 + bx + c = 0  
D = b^2 - 4a  
(-b +- sqrt(D)) / 2a  

In [41]:
D = time ** 2 - 4 * distance
D

1445336965368632

In [42]:
import math

t1 = math.ceil((time - math.sqrt(D)) / 2)
t2 = math.floor((time + math.sqrt(D)) / 2)
print(t1, t2)
print(t2 - t1 + 1)

5980954 43998540
38017587


## Day 07: Camel Cards

In [56]:
day = 7
day7 = dynamic_load(day)
input_path = get_input_path(day=day, test=None)

### part 1

In [57]:
df = day7.parse_input(input_path)

In [58]:
df = day7.add_rank(df)
df

hand,bid,hand_hex,option_counts,count1,count2,prio,rank
str,i64,i32,list[u32],u32,u32,i32,u32
"""4KTJ4""",575,318132,"[2, 1, … 0]",2,1,2,254
"""38T4K""",449,232013,"[1, 1, … 0]",1,1,1,25
"""4T437""",860,304183,"[2, 1, … 0]",2,1,2,248
"""55954""",240,350548,"[3, 1, … 0]",3,1,4,665
"""Q8K89""",150,822665,"[2, 1, … 0]",2,1,2,415
"""85838""",21,546872,"[3, 1, … 0]",3,1,4,703
"""68668""",464,427624,"[3, 2, … 0]",3,2,5,843
"""JQT77""",737,772727,"[2, 1, … 0]",2,1,2,397
"""99497""",155,627863,"[3, 1, … 0]",3,1,4,724


In [59]:
(
    df
    .select(
        (pl.col('rank') * pl.col('bid'))
    ).sum()
)

rank
i64
248179786
